In [1]:
import pandas as pd
import re

In [2]:
%run ../functions.ipynb

### Import data

In [4]:
data_path = '../../data/MDataFiles_Stage1/'

#raw data
rankings = pd.read_csv(data_path + 'MMasseyOrdinals.csv')
detailed_reg_season = pd.read_csv(data_path +'MRegularSeasonDetailedResults.csv')
team_coaches = pd.read_csv(data_path + 'MTeamCoaches.csv')
detailed_tourney_results = pd.read_csv(data_path + 'MNCAATourneyDetailedResults.csv')
conf_tourney = pd.read_csv(data_path + 'MConferenceTourneyGames.csv')
team_conferences = pd.read_csv(data_path + 'MTeamConferences.csv')
tourney_seeds = pd.read_csv(data_path + 'MNCAATourneySeeds.csv')
teams = pd.read_csv(data_path + 'MTeams.csv')

#processed data
all_games = pd.read_csv('../processing/output/all_games.csv')

#### Add efficiency metrics

In [5]:
all_games = get_def_off(all_games)

#### Add opponents nested efficiency metrics

A quick note on what these values represent:

We start by collecting the offensive and defensive efficiencies for each team at the point in the season when the game happened. Then we join in the opponents' offensive and defensive efficiencies.

Now we for every team we have oe and de as well as their opponent's oe and de at a point in time.

We can then take an average of opponents oe and de to get on average how good the opponents were.

Once we have that, we also have it for the opponents, so we can join that in as well.

Ultimately we can iteratively go as many layers deep as we want of opponents' opponents' opponents' etc. efficiency. This gives us measures for how well a team performed, as well as deep measures of strength of schedule. 

For example - Let's say Cuse has played Gtown, Duke, Lville 

I first calc Cuse's eff. 

Then join I can join in eff for Gtown Duke and Lville.

Then I can get the average eff of Gtown Duke and Lville so I know the opponent's eff

But, now in my dataset I know the opponent's eff for Gtown Duke and Lville

So for Cuse I can join that back in and know the avg opponents avg opponents eff. In other words, not only how tough has competition been against Duke/Lville/Gtown.

Then of course I can join that back in again get an avg of how tough the competition has been against the competition against Duke/Lville/Gtown



In [6]:
#sort values for rolling
all_games.sort_values(by = ['Season', 'Team1', 'DayNum'], inplace = True)
all_games.reset_index(drop=True, inplace = True)

#expanding roll to get avg oe and de for entire season
all_games['avg_oe'] = all_games.groupby(['Season', 'Team1'])['OffRtg'].transform(lambda x: x.shift(1).expanding().mean())
all_games['avg_de'] = all_games.groupby(['Season', 'Team1'])['DefRtg'].transform(lambda x: x.shift(1).expanding().mean())

#starting values for Team1's avg efficiency
oe_now = 'avg_oe'
de_now = 'avg_de'

for x in range(4):
    
    #create strings for opponents values
    oe_next = 'opp_' + oe_now
    de_next = 'opp_' + de_now 

    #create copy of data where T1 = T2 and change efficiencies names to opponent's
    cols = ['Season', 'DayNum', 'Team2', oe_next, de_next]
    all_games2 = all_games.rename(columns = \
                        {'Team1': 'Team2', 'Team2': 'Team1', oe_now: oe_next, de_now: de_next})
    #join in opponents stats
    all_games = all_games.merge(all_games2[cols], on = ['Season', 'DayNum', 'Team2'])

    #get rolling avg of opponent's stats
    all_games['avg_' + oe_next] = all_games.groupby(['Season', 'Team1'])[oe_next].transform(lambda x: x.shift(1).expanding().mean())
    all_games['avg_' + de_next] = all_games.groupby(['Season', 'Team1'])[de_next].transform(lambda x: x.shift(1).expanding().mean())
    
    #update variables to reflect new metrics 
    oe_now = 'avg_' + oe_next
    de_now = 'avg_' + de_next

#all_games = all_games.dropna()


#### Add seeds

In [7]:
tourney_seeds.rename(columns = {'TeamID': 'Team1'}, inplace = True)
join_key = ['Season', 'Team1']
all_games = all_games.merge(tourney_seeds[join_key + ['Seed']], on = join_key, how = 'left')
all_games['Seed'] = all_games['Seed'].fillna('9999').apply(lambda x: re.sub('[^0-9]','', x)).apply(int)


#### Create features for both teams in the game

In [8]:
cols = ['OffRtg', 'DefRtg', 'avg_oe', 'avg_de', 'opp_avg_oe',
       'opp_avg_de', 'avg_opp_avg_oe', 'avg_opp_avg_de', 'opp_avg_opp_avg_oe',
       'opp_avg_opp_avg_de', 'avg_opp_avg_opp_avg_oe',
       'avg_opp_avg_opp_avg_de', 'opp_avg_opp_avg_opp_avg_oe',
       'opp_avg_opp_avg_opp_avg_de', 'avg_opp_avg_opp_avg_opp_avg_oe',
       'avg_opp_avg_opp_avg_opp_avg_de', 'opp_avg_opp_avg_opp_avg_opp_avg_oe',
       'opp_avg_opp_avg_opp_avg_opp_avg_de',
       'avg_opp_avg_opp_avg_opp_avg_opp_avg_oe',
       'avg_opp_avg_opp_avg_opp_avg_opp_avg_de', 'Seed']

all_games = get_cols_for_both_teams(all_games, cols)

In [9]:
all_games['Seed_Diff'] = all_games['Team1_Seed'] - all_games['Team2_Seed']

In [10]:
all_games.to_csv('output/all_games.csv')